# Youtube Recommender System

### Group Meeting: 12/3/2019

In [497]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import pandas as pd
import nltk

from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report

import json

import warnings
warnings.filterwarnings('ignore')

In [426]:
us_video_data = pd.read_csv("dataset/USvideos.csv")
ca_video_data = pd.read_csv("dataset/CAvideos.csv")
gb_video_data = pd.read_csv("dataset/GBvideos.csv")

video_data = us_video_data.append(ca_video_data, ignore_index=True)
video_data = video_data.append(gb_video_data, ignore_index=True)
video_data.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [427]:
len(video_data)

120746

In [428]:
clean_vid_data = video_data.groupby(["title"]).first().reset_index()

In [429]:
clean_vid_data.head(5)

,title,video_id,trending_date,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,!! THIS VIDEO IS NOTHING BUT PAIN !! | Getting...,PNn8sECd7io,18.04.01,Markiplier,20,2018-01-03T19:33:53.000Z,"getting over it|""markiplier""|""funny moments""|""...",835930,47058,1023,8250,https://i.ytimg.com/vi/PNn8sECd7io/default.jpg,False,False,False,Getting Over It continues with RAGE BEYOND ALL...
1,"#1 Fortnite World Rank - 2,323 Solo Wins!",DvPW66IFhMI,18.09.03,AlexRamiGaming,20,2018-03-09T07:15:52.000Z,"PS4 Battle Royale|""PS4 Pro Battle Royale""|""Bat...",212838,5199,542,11,https://i.ytimg.com/vi/DvPW66IFhMI/default.jpg,False,False,False,Discord For EVERYONE - https://discord.gg/nhud...
2,"#1 Fortnite World Rank - 2,330 Solo Wins!",EXEaMjFeiEk,18.10.03,AlexRamiGaming,20,2018-03-10T06:26:17.000Z,"PS4 Battle Royale|""PS4 Pro Battle Royale""|""Bat...",200764,5620,537,45,https://i.ytimg.com/vi/EXEaMjFeiEk/default.jpg,False,False,False,Discord For EVERYONE - https://discord.gg/nhud...
3,#1 MOST ANTICIPATED VIDEO (Timber Frame House ...,bYvQmusLaxw,17.20.12,Pure Living for Life,24,2017-12-20T02:49:11.000Z,"timber frame|""timber framing""|""timber frame ra...",79152,7761,159,1965,https://i.ytimg.com/vi/bYvQmusLaxw/default.jpg,False,False,False,Shelter Institute: http://bit.ly/2iwXj8B\nFull...
4,#1 WORLD RANKED 1463 SOLO WINS! - FORTNITE BAT...,xQ4Q5b2WwO8,18.18.01,AlexRamiGaming,20,2018-01-17T18:00:05.000Z,"PS4 Battle Royale|""PS4 Pro Battle Royale""|""Bat...",541482,15430,891,40,https://i.ytimg.com/vi/xQ4Q5b2WwO8/default_liv...,False,False,False,►Twitter @AlexRamiGaming\n\n►Tips & Donations\...


In [430]:
len(clean_vid_data)

30626

## Part 1: Category Classification Model

In [405]:
def create_category_map():
    category_map = {}
    data = {}
    with open('dataset/US_category_id.json', 'r') as outfile:
        data = json.load(outfile)
        for item in data["items"]:
            category_map[item["id"]] = item["snippet"]["title"]
    return category_map
category_map = create_category_map()

In [406]:
category_map

{'1': 'Film & Animation',
 '2': 'Autos & Vehicles',
 '10': 'Music',
 '15': 'Pets & Animals',
 '17': 'Sports',
 '18': 'Short Movies',
 '19': 'Travel & Events',
 '20': 'Gaming',
 '21': 'Videoblogging',
 '22': 'People & Blogs',
 '23': 'Comedy',
 '24': 'Entertainment',
 '25': 'News & Politics',
 '26': 'Howto & Style',
 '27': 'Education',
 '28': 'Science & Technology',
 '29': 'Nonprofits & Activism',
 '30': 'Movies',
 '31': 'Anime/Animation',
 '32': 'Action/Adventure',
 '33': 'Classics',
 '34': 'Comedy',
 '35': 'Documentary',
 '36': 'Drama',
 '37': 'Family',
 '38': 'Foreign',
 '39': 'Horror',
 '40': 'Sci-Fi/Fantasy',
 '41': 'Thriller',
 '42': 'Shorts',
 '43': 'Shows',
 '44': 'Trailers'}

In [407]:
import re

def preprocess_text_df(text):
    text = text.apply(lambda x: x.lower())
    text = text.str.replace(r'[^a-zA-Z ]', '') #text.str.replace(r'[^\w\s]', '')
    return text

def preprocess_text(text):
    text = text.lower()
    regex = re.compile('[^a-zA-Z ]')
    text = regex.sub('', text)
    return text

In [408]:
preprocess_text("Dua Lipa - IDGAF (Official Music Video)")

'dua lipa  idgaf official music video'

In [409]:
def predict_category(X_train, y_train, X_test, vectorizer, clf):
    X_train = vectorizer.fit_transform(X_train)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(vectorizer.transform(X_test))
    return y_pred

In [ ]:
def predict_category_from_title(title):
    ....
    
    
    return category

In [431]:
clean_vid_data["title_clean"] = preprocess_text_df(clean_vid_data.title)
X_train, X_test, y_train, y_test = train_test_split(clean_vid_data["title_clean"], 
                                                    clean_vid_data['category_id'], 
                                                    test_size=0.1, 
                                                    shuffle=True
                                                    )
y_pred = predict_category(X_train, y_train, X_test, TfidfVectorizer(), SGDClassifier())
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.64      0.47      0.54       138
           2       0.78      0.57      0.66        37
          10       0.68      0.86      0.76       272
          15       0.45      0.31      0.37        29
          17       0.71      0.86      0.78       229
          19       0.70      0.78      0.74        18
          20       0.57      0.40      0.47        92
          22       0.57      0.29      0.38       313
          23       0.66      0.41      0.51       234
          24       0.62      0.76      0.68      1002
          25       0.60      0.62      0.61       335
          26       0.66      0.77      0.71       182
          27       0.42      0.23      0.30        79
          28       0.61      0.47      0.53        88
          29       0.00      0.00      0.00         3
          43       0.50      0.17      0.25        12

    accuracy                           0.63      3063
   macro avg       0.57   

/Users/frys/Documents/MIDS/w207/w207/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [167]:
X_test

780     black panther continues its reign breaks more ...
3275    lady gaga  the cure live at amas from joanne w...
3510             my thoughts about making the forbes list
1186    colin farrell dishes on rendezvous with girlfr...
3243                    la noire  nintendo switch trailer
                              ...                        
2813                                  its ok to not be ok
3936    new 2018 ipad announced 329 a10 fusion  apple ...
4510              roy moore  jeff sessions cold open  snl
2085                       google pixelbook 1000 of weird
3754                migos nicki minaj cardi b  motorsport
Name: title, Length: 646, dtype: object

In [46]:
y_pred

array([25,  1, 24, 26, 15, 10, 25,  1, 10, 23, 25, 25, 26, 24, 24,  1, 24,
       23, 22,  2, 26, 17, 10, 24, 25, 24, 24, 24, 28, 24, 10, 24, 24, 23,
       24, 24, 22, 25, 24, 23, 25, 27, 24, 17, 17, 24, 27, 24, 27, 17, 24,
       22, 23, 28, 24, 24, 20, 23, 24, 20, 22, 24, 17, 20, 22, 26, 23, 22,
       26, 24, 24, 27, 24, 10, 26, 15, 22, 24, 19, 25, 26, 10, 22, 28, 24,
       26, 10, 10, 24, 28, 27, 17, 24, 24, 24, 25, 24, 24, 26, 25, 26, 10,
       26, 24, 10, 17,  1, 10, 10, 24, 10, 22, 24, 27, 22, 25, 10, 24, 28,
       24, 22, 22,  1, 24, 25, 24, 15, 25, 20, 10, 24, 27, 28, 23, 24, 10,
       17, 17, 25, 24, 10, 24, 10, 10, 24, 17, 15, 24, 24, 24,  1, 27, 23,
        1,  1, 28, 22, 26, 23, 24, 28, 24, 10, 24, 22, 25, 17, 17, 25, 24,
        2, 24, 24,  1, 26, 25, 25, 22, 27, 17, 23, 27, 25, 24, 24, 23, 22,
       19, 26, 27, 26, 26, 27, 24, 17, 10, 22, 26, 23, 10, 17, 22,  1, 26,
       25, 27, 28, 24, 24, 28, 10, 27, 26, 17, 22, 25, 26, 24, 24, 24, 10,
       25, 26, 23, 24, 22

In [432]:
metrics.accuracy_score(y_test, y_pred)

0.6314071172053543

In [412]:
def category_group(num):
    category_number = num    
    category_name = category_map[str(num)]
    rslt_df = clean_vid_data[clean_vid_data['category_id'] == category_number] 
    return rslt_df.reset_index()

ann

In [413]:
def find_tags(vidtitle, rslt_df) : 
    row = rslt_df[rslt_df['title'] == vidtitle]
    tags = row['tags']
    list_of_tags = []
    for it in tags:
        ls = it.split('|')
        for l in ls:
            if l != '[none]':
                list_of_tags.append(l)    
    return list_of_tags#[0] if len(list_of_tags) else list_of_tags

## Part 2: Same Category Video Rankings

annabelle

### 2.1 Ranking Based on Title

In [435]:
video = clean_vid_data[["title", "category_id"]]
video["title_clean"] = preprocess_text_df(video.title)
video.head()

/Users/frys/Documents/MIDS/w207/w207/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,category_id,title_clean
0,!! THIS VIDEO IS NOTHING BUT PAIN !! | Getting...,20,this video is nothing but pain getting over...
1,"#1 Fortnite World Rank - 2,323 Solo Wins!",20,fortnite world rank solo wins
2,"#1 Fortnite World Rank - 2,330 Solo Wins!",20,fortnite world rank solo wins
3,#1 MOST ANTICIPATED VIDEO (Timber Frame House ...,24,most anticipated video timber frame house rai...
4,#1 WORLD RANKED 1463 SOLO WINS! - FORTNITE BAT...,20,world ranked solo wins fortnite battle roya...


In [484]:
#source: https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50
def get_jaccard_sim(a, str2): 
    #a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def get_jaccard_sim_list(list1, list2): 
    a = set(list1) 
    b = set(list2)
    c = a.intersection(b)
    if (len(a) + len(b) - len(c)) == 0:
        return 0
    return float(len(c)) / (len(a) + len(b) - len(c))

In [503]:
def find_similar_title(v):
    cleaned_title = preprocess_text(v)
    a = set(cleaned_title.split())
    
    category = video.loc[video.title.str.endswith(v)]["category_id"].values[0]
    video_category = video.loc[video['category_id']==category]
    
    score = []
    
    for t in video_category.title_clean:
        score += [get_jaccard_sim(a, t)]
    video_category["title_score"] = score
    
    video_category = video_category.loc[video_category['title_score'] < 1].sort_values(by=['title_score'], ascending=False)
    return video_category.reset_index().drop(columns=['index', 'title_clean'])[0:25]

#similar_video_titles = find_similar_title("#ProudToCreate: Pride 2018")
similar_video_titles = find_similar_title("Marshmello - FLY (Official Music Video)")

In [417]:
preprocess_text("Dua Lipa - IDGAF (Official Music Video)")

'dua lipa  idgaf official music video'

In [ ]:
video["title_clean"]

In [477]:
similar_video_titles.head(10)

,title,category_id,title_score
0,Marshmello - Blocks (Official Music Video),10,0.666667
1,Marshmello - You & Me (Official Music Video),10,0.571429
2,Marshmello - Love U (Official Music Video),10,0.571429
3,Marshmello - LoVe U (Official Music Video),10,0.571429
4,Marshmello - Take It Back (Official Music Video),10,0.500000
5,Marshmello & Anne-Marie - FRIENDS (Music Video...,10,0.444444
6,Marshmello x Lil Peep - Spotlight (Official Mu...,10,0.444444
7,Ghost - Rats (Official Music Video),10,0.428571
8,Justice - Stop (Official Music Video),10,0.428571
9,PENTAGON(펜타곤) - '빛나리(Shine)' Official Music Video,10,0.428571


### 2.2 Ranking Based on Tags

In [419]:
tags = clean_vid_data["tags"]
tags[1]

'PS4 Battle Royale|"PS4 Pro Battle Royale"|"Battle Royale Win"|"Fornite Duo"|"Fortnite Squads"|"solo"|"Fortnite"|"Umbrella"|"Fortnite Gameplay"|"Fortnite PS4"|"Fortnite Stream"|"Fortnite Game"|"Fortnite Tips"|"Fortnite Tricks"|"Fortnight"|"Fort Night"|"Fort Nite"|"fortnite br"|"fortnite pubg"|"fornite battle royale"|"fortnight br"|"fortnight"|"fortnite br solo"|"fortnite solo"|"fortnite new update"|"fortnite new city"|"fortnite patch"|"Fortnite #1 World Rank"'

In [420]:
def map_titles_to_tags(category):
    category_df = category_group(category)
    title_tags_dict = {}
    for i in range(len(category_df)):
        video_title = category_df["title"][i]
        video_tags = find_tags(video_title, category_df)
        title_tags_dict[video_title] = video_tags
    return title_tags_dict

In [58]:
# def find_category_tags(interested_category):
#     tags_df = pd.DataFrame()
#     title_tags_dict = {}
#     trimed_df = category_group(interested_category)
#     for i in range(len(trimed_df["tags"])):
        
#         video_title = trimed_df["title"][i]
#         ls = trimed_df["tags"][i]
#         individual_tags = ls.split("|")
#         for it in individual_tags[0:5]:
#             if it != '[none]':
#                 tags_df = tags_df.append({"Tag Name": it, "Video Title": video_title}, ignore_index=True)
#     return tags_df

In [502]:
def find_similar_tag_video(title):
    
    category = video.loc[video.title.str.endswith(title)]["category_id"].values[0]
    #category_df = video.loc[video['category_id']==category]
    
    category_df = category_group(category)
    #return category_df
    interested_tags = find_tags(title, category_df)
    category_df = category_df[["title"]]
#     print(category)
#     print(interested_tags)
    title_tag_mapping = map_titles_to_tags(category)
    score = []
    i = 0
    for title, tags in title_tag_mapping.items():
        score.append(get_jaccard_sim_list(interested_tags, tags))
    
    category_df["tag_score"] = score
    
    category_df = category_df.loc[category_df['tag_score'] > 0].loc[category_df['tag_score'] != 1].loc[category_df['tag_score'] != len(interested_tags)].sort_values(by = ['tag_score'], ascending=False)

    return category_df.reset_index().drop(columns=['index'])[0:25]

#similar_videos_tags = find_similar_tag_video("Calvin Harris, Dua Lipa - One Kiss (Lyric Video)")
similar_videos_tags = find_similar_tag_video("Dua Lipa - IDGAF (Official Music Video)")  
#similar_videos_tags = find_similar_tag_video("#ProudToCreate: Pride 2018")

In [465]:
similar_videos_tags.head(10)

,title,tag_score
0,Kendrick Lamar DAMN | Best Rap Album | 60th GR...,2.0
1,Don Diablo ft. Ansel Elgort - Believe | Lyric ...,2.0
2,Bruno Mars Wins Album Of The Year | Acceptance...,2.0
3,Azealia Banks - Anna Wintour,2.0
4,Dua Lipa - New Rules (Live at The BRIT Awards ...,2.0
5,Troye Sivan - Dance To This (Official Audio) f...,2.0
6,Bruno Mars Wins Record Of The Year | Acceptanc...,2.0
7,Who's Performing at the 2018 GRAMMYs?,2.0
8,Nina Nesbitt - Somebody Special (Official Video),1.0
9,Meghan Trainor - Can't Dance (Audio),1.0


In [454]:
category_df10 = category_group(10)
a0 = find_tags("Calvin Harris, Dua Lipa - One Kiss (Lyric Video)", category_df10)#"Dua Lipa - IDGAF (Official Music Video)", category_df10)
a0

['calvin harris',
 '"calvin harris one kiss"',
 '"calvin harris dua lipa one kiss"',
 '"dua lipa one kiss"',
 '"one kiss"',
 '"one kiss lyrics"',
 '"calvin harris one kiss lyrics"',
 '"calvin harris dua lipa one kiss live"',
 '"calvin harris dua lipa one kiss lyrics"',
 '"dua lipa one kiss lyrics"',
 '"dua lipa one kiss live"',
 '"calvin harris feels"',
 '"calvin harris summer"',
 '"calvin harris nuh ready"',
 '"calvin harris thinking about you"',
 '"calvin harris my way"',
 '"calvin harris feel so close"',
 '"calvin"',
 '"harris"']

In [340]:
a1 = find_tags("Azealia Banks - Anna Wintour", category_df10)
a1

['Azealia',
 '"Banks"',
 '"Anna"',
 '"Wintour"',
 '"eOne"',
 '"Music"',
 '"Dance"',
 '"Alternative/Indie"',
 '"Electronic"',
 '"Club/Dance"',
 '"Pop"']

In [339]:
get_jaccard_sim_list(a0, a1)

0.06060606060606061

In [342]:
a2 = find_tags("Dua Lipa - Golden Slumbers", category_df10)
a2

['dua lipa',
 '"golden slumbers"',
 '"xmas songs"',
 '"christmas music"',
 '"john lewis advert"',
 '"christmas"',
 '"festive"']

In [343]:
get_jaccard_sim_list(a0, a2)

0.03333333333333333

## Putting It All Together

In [498]:
def make_recommendations(title):
    
    top_titles = find_similar_title(title)
    top_tags = find_similar_tag_video(title)
    
    category = top_titles["category_id"][0]
    category_df = category_group(category)
    
    top_titles = top_titles.drop(columns=['category_id'])
    interested_tags = find_tags(title, category_df)
    cleaned_title = preprocess_text(title)
    title_set = set(cleaned_title.split())
    
    title_supplement_score = []
    for t in top_titles["title"]:
        tags = find_tags(t, category_df)
        title_supplement_score.append(get_jaccard_sim_list(interested_tags, tags))
    
    tags_supplement_score = []
    for t in top_tags["title"]:        
        tags_supplement_score.append(get_jaccard_sim(title_set, t))
    
    top_titles["tag_score"] = title_supplement_score
    top_tags["title_score"] = tags_supplement_score     
    
    combined_top = top_titles.append(top_tags, ignore_index = True)
    combined_top["Combined Score"] = np.array(combined_top["tag_score"]) +  np.array(combined_top["title_score"])
    
    combined_top = combined_top.sort_values(by = ['Combined Score'], ascending=False).reset_index().drop(columns=['index'])
    return combined_top

In [504]:
make_recommendations("Dua Lipa - IDGAF (Official Music Video)").head(10)

,tag_score,title,title_score,Combined Score
0,0.000000,"Calvin Harris, Dua Lipa - One Kiss (Official V...",0.400,0.400000
1,0.020833,Veno - Dolce (Official Music Video),0.375,0.395833
2,0.000000,Marshmello - Blocks (Official Music Video),0.375,0.375000
3,0.000000,Justice - Stop (Official Music Video),0.375,0.375000
4,0.000000,Marshmello - Fly (Official Music Video),0.375,0.375000
5,0.000000,Cam - Diane (Official Music Video),0.375,0.375000
6,0.000000,Marshmello - FLY (Official Music Video),0.375,0.375000
7,0.000000,DDG - Arguments (Official Music Video),0.375,0.375000
8,0.000000,Ghost - Rats (Official Music Video),0.375,0.375000
9,0.000000,PENTAGON(펜타곤) - '빛나리(Shine)' Official Music Video,0.375,0.375000


In [505]:
make_recommendations("#ProudToCreate: Pride 2018").head(10)

,tag_score,title,title_score,Combined Score
0,0.000000,LGBT Pride | Brooklyn Nine-Nine,0.200000,0.200000
1,0.073529,'Shoes Everywhere!' Out Of The Closet w/ Kimor...,0.000000,0.073529
2,0.000000,Dragon Ball Super Episode 122 English Subbed E...,0.071429,0.071429
3,0.049383,Top 10 Cringiest Moments from RuPaul's Drag Race,0.000000,0.049383
4,0.028169,Jessica Chastain Is Not Afraid to End Kenan Th...,0.000000,0.028169
5,0.027027,Queer Eye | Theme Song (All Things) Feat. Bett...,0.000000,0.027027
6,0.026667,Driver Disagrees With Jazz Over Raising Money ...,0.000000,0.026667
7,0.026667,Driver Disagrees With Jazz Over Raising Money ...,0.000000,0.026667
8,0.026667,Hateful Driver Verbally Attacks Jazz And Frien...,0.000000,0.026667
9,0.026316,Marvel Studios' Avengers: Infinity War - Offic...,0.000000,0.026316


In [506]:
make_recommendations("#1 Fortnite World Rank - 2,330 Solo Wins!").head(10)

,tag_score,title,title_score,Combined Score
0,1.000000,NEW UPDATE #1 WORLD RANKED FORTNITE SOLO PLAYE...,0.444444,1.444444
1,0.903226,"🔴 #1 World Ranked | 2,532 Solo Wins | Fortnite...",0.500000,1.403226
2,1.000000,"#1 World Ranked - 1,700 Solo Wins - New Minig...",0.333333,1.333333
3,0.903226,"FREE VBUCKS TODAY | FORTNITE WORLD RECORD 2,75...",0.400000,1.303226
4,0.903226,"FORTNITE WORLD RECORD 2,732 SOLO WINS - New Fo...",0.363636,1.266862
5,0.666667,"#1 World Record 3,225 Solo Wins | Fortnite Liv...",0.500000,1.166667
6,0.628571,"#1 WORLD RECORD 3,288 SOLO WINS",0.500000,1.128571
7,0.628571,"#1 World Record 3,359 Solo Wins | Fortnite Liv...",0.500000,1.128571
8,0.666667,#1 WORLD RANKED 1463 SOLO WINS! - FORTNITE BAT...,0.400000,1.066667
9,0.903226,"FREE VBUCKS TODAY | FORTNITE WORLD RECORD 2,75...",0.000000,0.903226
